In [1]:
import argparse
import collections
import torch
import numpy as np
import model.loss as module_loss
import model.metric as module_metric
from parse_config import ConfigParser
from utils.util import create_model, create_dataloader, create_trainer


In [2]:
# fix random seeds for reproducibility
SEED = 125
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [3]:
args = argparse.ArgumentParser(description='Emotion Reasoning in Daily Life')
args.add_argument('-c', '--config', default='train.json', type=str,
                  help='config file path (default: None)')
args.add_argument('-r', '--resume', default=None, type=str,
                  help='path to latest checkpoint (default: None)')
args.add_argument('-d', '--device', default=None, type=str,
                  help='indices of GPUs to enable (default: all)')

# custom cli options to modify configuration from default values given in json file.
CustomArgs = collections.namedtuple('CustomArgs', 'flags type target')
options = [
    CustomArgs(['--lr', '--learning_rate'], type=float, target='optimizer;args;lr'),
    CustomArgs(['--bs', '--batch_size'], type=int, target='data_loader;args;batch_size')
]

In [4]:
config = ConfigParser.from_args(args, options)

jupyter


In [5]:
config._config['name'] += "_add_text"

In [6]:
print(config._config)

OrderedDict([('name', 'Train_9_2022_add_text'), ('message', 'MEmoR Baseline'), ('anno_file', 'data/anno.json'), ('data_file', 'data/data.json'), ('id_file', 'data/train_id.txt'), ('emo_type', 'primary'), ('speakers', ['sheldon', 'leonard', 'howard', 'rajesh', 'penny', 'bernadette', 'amy', 'others']), ('audio', OrderedDict([('feature_file', 'data/features/audio_features.json'), ('feature_dim', 6373)])), ('text', OrderedDict([('bert_model_name', 'bert-large-uncased'), ('feature_file', 'data/features/text_features.json'), ('feature_dim', 1024)])), ('visual', OrderedDict([('frame_path', 'data/clips_frames'), ('faces_feature_dir', 'data/features/visual_features/face_features'), ('faces_names_dir', 'data/features/visual_features/face_names'), ('obj_feature_dir', 'data/features/visual_features/object_features'), ('env_feature_dir', 'data/features/visual_features/environment_features'), ('dim_env', 2048), ('dim_obj', 1230), ('dim_face', 1024)])), ('personality', OrderedDict([('used', True), ('

In [7]:
logger = config.get_logger('train')

In [8]:
from data_loader_add.data_loaders_add import AddMEmoRDataLoader

In [9]:
# setup data_loader instances
data_loader = AddMEmoRDataLoader(config)
valid_data_loader = data_loader.split_validation()

Initializing VisualFeatureExtractor...
Initializing AudioFeatureExtracor...
Initializing TextFeatureExtractor...
Initializing PersonalityFeatureExtractor...
Initializing Add_TextFeatureExtractor...


  0%|▏                                                                                                                                | 11/6829 [00:00<01:03, 107.54it/s]

vectorize features.....


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6829/6829 [01:03<00:00, 108.37it/s]


In [10]:
from model_add.model_add import AddAMER

In [11]:
# build model architecture, then print to console
model = AddAMER(config)

In [12]:
# get function handles of loss and metrics
criterion = getattr(module_loss, config['loss'])
metrics = [getattr(module_metric, met) for met in config['metrics']]

In [13]:
from utils.util import create_trainer

In [14]:
trainer = create_trainer(model, criterion, metrics, logger,
                  config=config,
                  data_loader=data_loader,
                  valid_data_loader=valid_data_loader
                  )

trainer [MEmoRTrainer] was created
AddAMER(
  (attn): ScaledDotProductAttention(
    (dropout): Dropout(p=0, inplace=False)
  )
  (enc_v): Sequential(
    (0): Linear(in_features=4302, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=384, bias=True)
    (3): ReLU()
    (4): Linear(in_features=384, out_features=256, bias=True)
  )
  (enc_a): Sequential(
    (0): Linear(in_features=6373, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=256, bias=True)
  )
  (enc_t): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=192, bias=True)
  )
  (enc_t_add): Sequential(
    (0): Linear(in_features=24, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=64, bias=True)
  )
  (enc_p): Sequential(
    (0): Linear(in

In [15]:
trainer.train()

ValueError: too many values to unpack (expected 12)